# Setup

In [ ]:
import os
import logging as log
from lisa.utils import setup_logging
setup_logging(level=log.ERROR)

from lisa.trace import Trace
from lisa.wa import WAOutput
from lisa.stats import Stats
from lisa.datautils import series_mean
from pandas import DataFrame
import pandas as pd
import scipy as sp
import numpy as np
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import holoviews as hv
from holoviews import opts
from bokeh.themes import built_in_themes
from tabulate import tabulate

from wp.notebook import WorkloadNotebookAnalysis, trim_wa_path, ptable

hv.extension('bokeh')
hv.renderer('bokeh').theme = built_in_themes['dark_minimal']
hv.renderer('bokeh').webgl = True
pio.templates.default = "plotly"
pio.templates.default = "plotly_dark"

color_cycle = hv.Cycle(['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52'])

opts.defaults(
    opts.Curve(tools=['hover'], show_grid=True, color=color_cycle, muted_alpha=0),
    opts.Table(bgcolor='black')
)

## Runs

In [ ]:
jank = WorkloadNotebookAnalysis('/home/kajpuc01/power/pixel6/jankbench', [
    'jankbench_base-wo-uclamp_10_2307',
    'jankbench_fix-wo-uclamp_10_2307',
    'jankbench_runnable-wo-uclamp_10_2307',
])

jank.show()

jank.analysis['jank'] = pd.concat([wa_output['jankbench'].df for wa_output in jank.wa_outputs])
jank.analysis['jank']['wa_path'] = jank.analysis['jank']['wa_path'].map(trim_wa_path)

display(jank.analysis['jank'])

# Benchmark scores

## Max frame durations

In [ ]:
jank.analysis['max_duration'] = jank.analysis['jank'].query("variable == 'total_duration'")[["wa_path", "iteration", "value"]].groupby(["wa_path"]).max().reset_index()
jank.analysis['max_duration']['variable'] = 'max_duration'
jank.summary['max_duration'] = jank.plot_gmean_bars(jank.analysis['max_duration'], x='variable', y='value', title='jankbench max frame duration', width=1000, height=600)

## Line plot - frame duration

In [ ]:
jank.analysis['mean_durations'] = jank.analysis['jank'].query("variable == 'total_duration'")[["wa_path", "iteration", "value"]].groupby(["wa_path", "iteration"]).agg(lambda x: series_mean(x)).reset_index()
jank.analysis['mean_durations']['variable'] = 'mean_duration'

ds = hv.Dataset(jank.analysis['mean_durations'], ['iteration', hv.Dimension('wa_path', values=jank.wa_paths)], 'value')
layout = ds.to(hv.Curve, 'iteration', 'value').overlay('wa_path').opts(legend_position='bottom').opts(shared_axes=False, title='Jankbench mean frame duration per-iteration')
layout.opts(
    opts.Curve(height=600, width=1500, axiswise=True, shared_axes=False),
)
layout

## Overall frame durations

In [ ]:
jank.summary['mean_duration'] = jank.plot_gmean_bars(jank.analysis['mean_durations'], x='variable', y='value', title='jankbench gmean frame duration', width=1000, height=600)

## Frame duration histogram

In [ ]:
fig = px.histogram(jank.analysis['jank'].query("variable == 'total_duration'"), x='value', color='wa_path', barmode='group', nbins=40, height=800, title='Jankench frame duration histogram')
fig.show(renderer='iframe')

## Frame duration ecdf

In [ ]:
fig = px.ecdf(jank.analysis['jank'].query("variable == 'total_duration'"), x='value', color='wa_path', height=800, title='')
fig.show()

## Overall jank percentage

In [ ]:
jank.analysis['jank_percs'] = jank.analysis['jank'].query("variable == 'jank_frame'").groupby(['wa_path', 'iteration']).size().reset_index().rename(columns={0:'count'})
jank.analysis['jank_percs']['jank_count'] = jank.analysis['jank'].query("variable == 'jank_frame' and value == 1.0").groupby(['wa_path', 'iteration']).size().reset_index().rename(columns={0:'count'})['count']
jank.analysis['jank_percs']['perc'] = round(jank.analysis['jank_percs']['jank_count'] / jank.analysis['jank_percs']['count'] * 100, 2)

ds = hv.Dataset(jank.analysis['jank_percs'], ['iteration', hv.Dimension('wa_path', values=jank.wa_paths)], 'perc')
layout = ds.to(hv.Curve, 'iteration', 'perc').overlay('wa_path').opts(legend_position='bottom').opts(shared_axes=False, title='Jankbench jank percentage per-iteration')
layout.opts(
    opts.Curve(height=600, width=1500, axiswise=True, shared_axes=False),
)
layout

In [ ]:
jank.analysis['jank_percs']['value'] = jank.analysis['jank_percs']['perc']
jank.analysis['jank_percs']['variable'] = 'jank_perc'
jank.summary['jank_percs'] = jank.plot_gmean_bars(jank.analysis['jank_percs'][['wa_path', 'iteration', 'value', 'variable']], x='variable', y='value', title='jankbench gmean jank percentage', width=1000, height=600)

In [ ]:
ds = hv.Dataset(jank.results, ['iteration', hv.Dimension('wa_path', values=jank.wa_paths), 'test_name', 'metric'], 'value')
layout = ds.to(hv.Curve, 'iteration', 'value').overlay('wa_path').opts(legend_position='bottom').layout('test_name').opts(shared_axes=False, title='Jankbench metric per-iteration').cols(3)
layout.opts(
    opts.Curve(height=600, width=500, axiswise=True, shared_axes=False),
)
layout

## Bar plot - jank percentage

In [ ]:
jank.plot_gmean_bars(jank.results.query("metric == 'jank_p'"), x='stat', y='value', facet_col='test_name', facet_col_wrap=4, title='jankbench gmean jank percentage', width=1600, height=1000)

## Bar plot - mean duration

In [ ]:
jank.plot_gmean_bars(jank.results.query("metric == 'mean'"), x='stat', y='value', facet_col='test_name', facet_col_wrap=4, title='jankbench gmean frame duration', width=1600, height=1000)

# Overutilized

In [ ]:
def postprocess_overutil(df):
    df['time'] = round(df['time'], 2)
    df['total_time'] = round(df['total_time'], 2)
    return df

jank.load_combined_analysis('overutilized.pqt', postprocess=postprocess_overutil)
jank.load_combined_analysis('overutilized_mean.pqt')
ptable(jank.analysis['overutilized_mean'])

## Line plot

In [ ]:
jank.plot_lines_px(jank.analysis['overutilized'], y='percentage', title='Overutilized percentage per-iteration')

# Perf

## Line plot

In [ ]:
metrics = ['cpu-migrations', 'context-switches', 'stalled-cycles-backend', 'page-faults', 'major-faults', 'cache-misses', 'instructions', 'cpu-cycles', 'cpu-clock']
ds = hv.Dataset(df_perf, ['iteration', hv.Dimension('wa_path', values=wa_paths), hv.Dimension('metric', values=metrics)], 'value')
layout = ds.select(metric=metrics).to(hv.Curve, 'iteration', 'value').overlay('wa_path').opts(legend_position='bottom').layout('metric').opts(shared_axes=False, title='Perf counters').cols(3)
layout.opts(
    opts.Curve(width=600, height=340),
    opts.Overlay(legend_position='bottom'),
)
layout

## Bar plot

In [ ]:
metrics = ['cpu-migrations', 'context-switches', 'stalled-cycles-backend', 'page-faults', 'major-faults', 'minor-faults', 'cache-misses', 'instructions', 'cpu-cycles', 'cpu-clock']
plot_gmean_bars(df_perf.query("metric in @metrics"), x='stat', y='value', facet_col='metric', facet_col_wrap=5, title='gmean perf counters', width=1900, height=900)

# Idle residency

In [ ]:
jank.load_combined_analysis('idle_residency.pqt')
jank.analysis['idle_residency'] = jank.analysis['idle_residency'].groupby(['wa_path', 'cluster', 'idle_state'], sort=False).mean().reset_index()[['wa_path', 'cluster', 'idle_state', 'time']]
jank.analysis['idle_residency']['time'] = round(jank.analysis['idle_residency']['time'], 2)

display(jank.analysis['idle_residency'])

## Bar plot

In [ ]:
fig = px.bar(jank.analysis['idle_residency'], x='idle_state', y='time', color='wa_path', facet_col='cluster', barmode='group', text=jank.analysis['idle_residency']['time'], width=1800, height=600, title='Idle state residencies')
fig.update_traces(textposition='outside')
fig.show(renderer='iframe')

# Idle misses

In [ ]:
def load_idle_misses(analysis):
    def load_idle_miss_df(analysis_dir):
        try:
            wakeup_count = len(pd.read_parquet(os.path.join(analysis_dir, 'cpu_idle.pqt')).query("state == -1"))
            misses = pd.read_parquet(os.path.join(analysis_dir, 'cpu_idle_miss_counts.pqt')).groupby(['wa_path', 'kernel', 'cluster', 'below']).sum().reset_index()
            misses['count_perc'] = round(misses['count'] / wakeup_count * 100, 3)
            return misses
        except FileNotFoundError as e:
            print(e)

    return pd.concat([load_idle_miss_df(os.path.join(analysis.benchmark_path, benchmark_dir, 'analysis')) for benchmark_dir in analysis.benchmark_dirs])


jank.analysis['cpu_idle_misses'] = load_idle_misses(jank)
jank.analysis['cpu_idle_misses']['type'] = jank.analysis['cpu_idle_misses']['below'].replace(0, 'too deep').replace(1, 'too shallow')
jank.analysis['cpu_idle_misses']['wa_path'] = jank.analysis['cpu_idle_misses']['wa_path'].map(trim_wa_path)

display(jank.analysis['cpu_idle_misses'])

## Bar plot

In [ ]:
ptable(jank.analysis['cpu_idle_misses'].groupby(['wa_path', 'type']).sum().reset_index()[['wa_path', 'type', 'count_perc']])
fig = px.bar(jank.analysis['cpu_idle_misses'], x='type', y='count_perc', color='wa_path', facet_col='cluster', barmode='group', text=jank.analysis['cpu_idle_misses']['count_perc'], width=1800, height=600, title='CPUIdle misses as percentage of all wakeups')
fig.show(renderer='iframe')

# Power usage

In [ ]:
def postprocess_pixel6_emeter_means(df):
    df_total = df.groupby(['wa_path', 'kernel', 'iteration']).sum().reset_index()
    df_total['channel'] = 'Total'

    df_cpu_total = df.query("channel.str.startswith('CPU')").groupby(['wa_path', 'kernel', 'iteration']).sum().reset_index()
    df_cpu_total['channel'] = 'CPU'
    return pd.concat([df, df_cpu_total, df_total])[['wa_path', 'kernel', 'iteration', 'channel', 'power']]


jank.load_combined_analysis('pixel6_emeter.pqt')
jank.load_combined_analysis('pixel6_emeter_mean.pqt', postprocess=postprocess_pixel6_emeter_means)

display(jank.analysis['pixel6_emeter_mean'])

## Line plot

In [ ]:
jank.plot_lines_px(jank.analysis['pixel6_emeter_mean'], y='power', facet_col='channel', facet_col_wrap=3, height=1000, title='Mean power usage across iterations [mW]')

## Bar plot

In [ ]:
jank.summary['power_usage'] = jank.plot_gmean_bars(jank.analysis['pixel6_emeter_mean'].rename(columns={'power':'value'}), x='channel', y='value', facet_col='metric', facet_col_wrap=5, title='Gmean power usage [mW]', width=1800, height=600, include_total=True, include_columns=['channel'])

# Energy estimate

In [ ]:
jank.load_combined_analysis('energy_estimate_mean.pqt')
jank.analysis['energy_estimate_melt'] = pd.melt(jank.analysis['energy_estimate_mean'], id_vars=['iteration', 'wa_path'], value_vars=['little', 'mid', 'big', 'total']).rename(columns={'variable':'cluster'})

## Line plot

In [ ]:
jank.plot_lines_px(jank.analysis['energy_estimate_melt'], y='value', facet_col='cluster', title='Mean energy estimate across iterations [bW]')

## Bar plot

In [ ]:
jank.summary['energy_estimate'] = jank.plot_gmean_bars(jank.analysis['energy_estimate_melt'], x='cluster', y='value', facet_col='metric', facet_col_wrap=5, title='Gmean energy estimate [bW]', width=1900, height=600, include_columns=['cluster'], order_cluster=True, include_total=True)

# Thermal

In [ ]:
def preprocess_thermal(df):
    return df.groupby(['iteration', 'kernel', 'wa_path']).mean().reset_index()

def postprocess_thermal(df):
    for col in [c for c in df.columns if c not in ['time', 'iteration', 'kernel', 'wa_path']]:
        df[col] = df[col] / 1000
    df = round(df, 2)
    return df

jank.load_combined_analysis('thermal.pqt', preprocess=preprocess_thermal, postprocess=postprocess_thermal)
jank.analysis['thermal_melt'] = pd.melt(jank.analysis['thermal'], id_vars=['iteration', 'wa_path', 'kernel'], value_vars=['little', 'mid', 'big']).rename(columns={'variable':'cluster'})
display(jank.analysis['thermal'])

## Line plot

In [ ]:
ds = hv.Dataset(jank.analysis['thermal_melt'], ['iteration', hv.Dimension('wa_path', values=jank.wa_paths), hv.Dimension('cluster', values=jank.CLUSTERS)], 'value')
layout = ds.to(hv.Curve, 'iteration', 'value').overlay('wa_path').opts(legend_position='bottom').layout('cluster').opts(title='Mean cluster temperature across iterations')
layout.opts(
    opts.Curve(width=600, height=600, ylabel='temperature'),
)
layout

## Bar plot

In [ ]:
jank.summary['thermal'] = jank.plot_gmean_bars(jank.analysis['thermal_melt'], x='cluster', y='value', facet_col='metric', facet_col_wrap=2, title='Gmean temperature', width=1800, height=600, order_cluster=True, include_columns=['cluster'])

# Frequency

In [ ]:
def postprocess_freq(df):
    df['unit'] = 'MHz'
    df['metric'] = 'frequency'
    df['order'] = df['cluster'].replace('little', 0).replace('mid', 1).replace('big', 2)
    return df.sort_values(by=['iteration', 'order']).rename(columns={'frequency':'value'})

jank.load_combined_analysis('freqs_mean.pqt', postprocess=postprocess_freq)
display(jank.analysis['freqs_mean'].head())

## Line plot

In [ ]:
ds = hv.Dataset(jank.analysis['freqs_mean'], ['iteration', hv.Dimension('wa_path', values=jank.wa_paths), hv.Dimension('cluster', values=jank.CLUSTERS)], 'value')
layout = ds.to(hv.Curve, 'iteration', 'value').overlay('wa_path').opts(legend_position='bottom').layout('cluster').opts(title='Mean cluster frequency across iterations')
layout.opts(
    opts.Curve(width=600, height=600, ylabel='MHz'),
)
layout

## Bar plot

In [ ]:
jank.summary['frequency'] = jank.plot_gmean_bars(jank.analysis['freqs_mean'], x='metric', y='value', facet_col='cluster', facet_col_wrap=3, title='Gmean frequency per cluster', width=1800, height=600, order_cluster=True, include_columns=['cluster'])

# CFS signals

In [ ]:
jank.load_combined_analysis('sched_pelt_cfs_mean.pqt')
jank.analysis['sched_pelt_cfs_melt'] = pd.melt(jank.analysis['sched_pelt_cfs_mean'], id_vars=['iteration', 'wa_path', 'kernel', 'cluster'], value_vars=['util', 'load'])

## Line plot

In [ ]:
signals = ['util', 'load']
ds = hv.Dataset(jank.analysis['sched_pelt_cfs_mean'], ['iteration', hv.Dimension('wa_path', values=jank.wa_paths), hv.Dimension('cluster', values=jank.CLUSTERS)], signals)
layout = hv.Layout([ds.to(hv.Curve, 'iteration', signal).overlay('wa_path').opts(legend_position='bottom').layout('cluster').opts(title='Mean cluster ' + signal, framewise=True) for signal in signals]).cols(1)
layout.opts(
    opts.Curve(width=800, height=400, framewise=True),
)
layout

## Bar plot

In [ ]:
jank.summary['cfs_signals'] = jank.plot_gmean_bars(jank.analysis['sched_pelt_cfs_melt'], x='cluster', y='value', facet_col='variable', facet_col_wrap=1, title='Gmean cfs signals', width=1900, height=1000, order_cluster=True, include_columns=['cluster'])

# Wakeup latency - tasks

In [ ]:
def postprocess_wakeup_latency_mean(df):
    df = df.rename(columns={'wakeup_latency':'value'})
    df['order'] = df['wa_path'].map(lambda x: jank.wa_paths.index(x))
    df['unit'] = 'x'
    return df

jank.load_combined_analysis('wakeup_latency_mean.pqt', postprocess=postprocess_wakeup_latency_mean)

def postprocess_wakeup_latency(df):
    df['order'] = df['wa_path'].map(lambda x: jank.wa_paths.index(x))
    df['cluster'] = df['cpu'].copy().apply(lambda c: 'little' if c < 4 else 'big' if c > 5 else 'mid')
    df['order_cluster'] = df['cluster'].map(lambda x: jank.CLUSTERS.index(x))
    df['target_cluster'] = df['target_cpu'].copy().apply(lambda c: 'little' if c < 4 else 'big' if c > 5 else 'mid')
    df['order_target_cluster'] = df['target_cluster'].map(lambda x: jank.CLUSTERS.index(x))
    return df

jank.load_combined_analysis('wakeup_latency.pqt', postprocess=postprocess_wakeup_latency)

display(jank.analysis['wakeup_latency'])

## Line plot

In [ ]:
jank.plot_lines_px(jank.analysis['wakeup_latency_mean'], facet_col='comm', facet_col_wrap=2, height=1000, title='Task wakeup latencies across iterations')

## Bar plot

In [ ]:
jank.summary['wakeup_latency'] = jank.plot_gmean_bars(jank.analysis['wakeup_latency_mean'], x='metric', y='value', facet_col='comm', facet_col_wrap=2, title='Gmean task wakeup latency', table_sort=['comm', 'kernel'], gmean_round=0, width=1900, height=1000)

## Quantiles

In [ ]:
jank.analysis['wakeup_latency_quantiles'] = jank.analysis['wakeup_latency'].groupby(['comm', 'wa_path', 'iteration']).quantile([0.9, 0.95, 0.99]).reset_index()[['comm', 'wa_path', 'level_3', 'iteration', 'wakeup_latency', 'order']].rename(columns={'level_3':'quantile'}).sort_values(by=['comm', 'order'])
jank.analysis['wakeup_latency_quantiles']

In [ ]:
jank.plot_gmean_bars(jank.analysis['wakeup_latency_quantiles'].rename(columns={'wakeup_latency':'value'}), x='quantile', y='value', facet_col='comm', facet_col_wrap=1, title='Gmean latency quantile', width=1900, include_columns=['quantile'], table_sort=['quantile', 'comm'], height=1200, gmean_round=0)

# Wakeup latency - cgroups

In [ ]:
def postprocess_cgroup_latency(df):
        df = df.rename(columns={'wakeup_latency':'value'})
        df['order'] = df['wa_path'].map(lambda x: jank.wa_paths.index(x))
        return df

jank.load_combined_analysis('wakeup_latency_cgroup.pqt', postprocess=postprocess_cgroup_latency)
jank.analysis['wakeup_latency_cgroup_mean'] = jank.analysis['wakeup_latency_cgroup'].groupby(["wa_path", "cgroup", "iteration", "order"]).agg(lambda x: series_mean(x)).reset_index().sort_values(by=["order", "cgroup", "iteration"])[['wa_path', 'cgroup', 'iteration', 'value', 'order']]
jank.analysis['wakeup_latency_cgroup_mean'].head()

## Line plot

In [ ]:
fig = px.line(jank.analysis['wakeup_latency_cgroup_mean'], x='iteration', y='value', color='wa_path', facet_col='cgroup', facet_col_wrap=3, height=600, title='cgroup wakeup latencies across iterations')
fig.update_yaxes(matches=None)
fig.show()

## Bar plot

In [ ]:
jank.plot_gmean_bars(jank.analysis['wakeup_latency_cgroup_mean'], x='metric', y='value', facet_col='cgroup', title='Gmean task wakeup latency per-cgroup', include_columns=['cgroup'], table_sort=['cgroup'], gmean_round=0, width=1800, height=600)

## Quantiles

In [ ]:
jank.analysis['wakeup_latency_cgroup_quantiles'] = jank.analysis['wakeup_latency_cgroup'].groupby(['cgroup', 'wa_path', 'iteration']).quantile([0.9, 0.95, 0.99]).reset_index()[['cgroup', 'wa_path', 'level_3', 'iteration', 'value', 'order']].rename(columns={'level_3':'quantile'}).sort_values(by=['cgroup', 'order'])
jank.analysis['wakeup_latency_cgroup_quantiles']

In [ ]:
jank.plot_gmean_bars(jank.analysis['wakeup_latency_cgroup_quantiles'], x='quantile', y='value', facet_col='cgroup', facet_col_wrap=1, title='Gmean latency quantile per-cgroup', include_columns=['cgroup', 'quantile'], table_sort=['quantile', 'cgroup'], width=1900, height=1400, gmean_round=0)

# CPU residency - tasks

In [ ]:
tasks_important = ['RenderThread', 'droid.benchmark', 'surfaceflinger']

def postprocess_task_residency_cpu_total(df):
    df = df.rename(columns={'Total':'total'})
    return df

jank.load_combined_analysis('tasks_residency_cpu_total.pqt', postprocess=postprocess_task_residency_cpu_total)
jank.analysis['task_residency_cpu_total_melt'] = pd.melt(jank.analysis['tasks_residency_cpu_total'], id_vars=['iteration', 'wa_path', 'kernel'], value_vars=jank.CPUS).rename(columns={'variable':'cluster'})
jank.analysis['task_residency_cpu_total_cluster_melt'] = pd.melt(jank.analysis['tasks_residency_cpu_total'], id_vars=['iteration', 'wa_path', 'kernel'], value_vars=jank.CLUSTERS_TOTAL).rename(columns={'cpu':'cluster'})

def postprocess_task_residency_total(df):
    df = df.rename(columns={'Total':'total'}).query("comm in @tasks_important")
    return df

jank.load_combined_analysis('tasks_residency_total.pqt', postprocess=postprocess_task_residency_total)
jank.analysis['tasks_residency_total'] = jank.analysis['tasks_residency_total'].sort_values(by=['wa_path', 'kernel', 'iteration'])
jank.analysis['task_residency_total_melt'] = pd.melt(jank.analysis['tasks_residency_total'], id_vars=['iteration', 'wa_path', 'kernel', 'comm'], value_vars=jank.CPUS).rename(columns={'variable':'cluster'})
jank.analysis['task_residency_total_cluster_melt'] = pd.melt(jank.analysis['tasks_residency_total'], id_vars=['iteration', 'wa_path', 'kernel', 'comm'], value_vars=jank.CLUSTERS_TOTAL).rename(columns={'cpu':'cluster'})

def postprocess_task_residency(df):
    df = df.rename(columns={'Total':'total'}).query("comm in @tasks_important")
    return df

jank.load_combined_analysis('tasks_residency.pqt', postprocess=postprocess_task_residency)
jank.analysis['tasks_residency_cluster_melt'] = pd.melt(jank.analysis['tasks_residency'], id_vars=['iteration', 'wa_path', 'kernel', 'comm'], value_vars=jank.CLUSTERS_TOTAL).rename(columns={'cpu':'cluster'})

display(jank.analysis['tasks_residency_cpu_total'])

jank.analysis['task_residency_total_melt']

## Clusters - Line plot

In [ ]:
ds = hv.Dataset(jank.analysis['task_residency_cpu_total_cluster_melt'], ['iteration', hv.Dimension('wa_path', values=jank.wa_paths), hv.Dimension('cluster', values=jank.CLUSTERS_TOTAL)], 'value')
layout = ds.to(hv.Curve, 'iteration', 'value').overlay('wa_path').opts(legend_position='bottom').layout('cluster').cols(4).opts(title='Mean cluster CPU residency')
layout.opts(
    opts.Curve(width=500, height=600, ylabel='time'),
)
layout

## Clusters - Bar plot

In [ ]:
jank.plot_gmean_bars(jank.analysis['task_residency_cpu_total_cluster_melt'], x='cluster', y='value', facet_col='metric', facet_col_wrap=2, title='Gmean cluster CPU residency', include_columns=['cluster'], width=1800, height=600, order_cluster=True, include_total=True)

## Clusters - Per-task bar plot

In [ ]:
jank.summary['task_cpu_residency'] = jank.plot_gmean_bars(jank.analysis['task_residency_total_cluster_melt'], x='cluster', y='value', facet_col='comm', facet_col_wrap=1, title='Gmean cluster task CPU residency', include_columns=['cluster'], width=1800, height=1200, order_cluster=True, include_total=True)

## CPUs - Line plot

In [ ]:
ds = hv.Dataset(jank.analysis['task_residency_total_melt'], ['iteration', hv.Dimension('wa_path', values=jank.wa_paths), hv.Dimension('cpu', values=jank.CPUS), 'comm'], 'value')
layout = ds.to(hv.Curve, 'iteration', 'value').overlay('wa_path').opts(legend_position='bottom').layout('cpu').cols(3).opts(title='Mean CPU residency', framewise=True)
layout.opts(
    opts.Curve(width=600, height=600, ylabel='time'),
)
layout

## CPUs - Bar plot

In [ ]:
jank.plot_gmean_bars(jank.analysis['task_residency_total_melt'], x='cpu', y='value', facet_col='comm', facet_col_wrap=1, title='Gmean CPU residency', width=1900, height=1200)

# CPU residency - cgroups

In [ ]:
def postprocess_cgroup_task_residency(df):
    df = df.rename(columns={'Total':'total'})[['wa_path', 'cgroup', 'iteration', 'total', 'little', 'mid', 'big'] + jank.CPUS]
    return df

jank.load_combined_analysis('tasks_residency_cgroup_total.pqt', postprocess=postprocess_cgroup_task_residency)
jank.analysis['cgroup_residency_total_melt'] = pd.melt(jank.analysis['tasks_residency_cgroup_total'], id_vars=['iteration', 'wa_path', 'cgroup'], value_vars=jank.CPUS).rename(columns={'variable':'cluster'})
jank.analysis['cgroup_residency_total_cluster_melt'] = pd.melt(jank.analysis['tasks_residency_cgroup_total'], id_vars=['iteration', 'wa_path', 'cgroup'], value_vars=jank.CLUSTERS_TOTAL).rename(columns={'cpu':'cluster'})

display(jank.analysis['tasks_residency_cgroup_total'].head())

## Clusters - Bar plot

In [ ]:
jank.plot_gmean_bars(jank.analysis['cgroup_residency_total_cluster_melt'], x='cluster', y='value', facet_col='cgroup', facet_col_wrap=1, title='Gmean cluster CPU residency per-cgroup', width=1800, height=1100, include_columns=['cgroup', 'cluster'], table_sort=['cgroup', 'cluster'], order_cluster=True, include_total=True)

## CPUs - Bar plot

In [ ]:
jank.plot_gmean_bars(jank.analysis['cgroup_residency_total_melt'], x='cpu', y='value', facet_col='cgroup', facet_col_wrap=1, title='Gmean cgroup CPU residency', width=2000, height=1300, include_columns=['cgroup', 'cpu'])

# Summary - TLDR

In [ ]:
summary_parts = []

if 'mean_duration' in jank.summary:
    summary_mean_durations = jank.summary['mean_duration'].copy()
    summary_mean_durations['perc_diff'] = summary_mean_durations['perc_diff'].apply(lambda s: f"({s})")
    summary_mean_durations['value'] = summary_mean_durations['value'] + " " + summary_mean_durations['perc_diff']
    summary_mean_durations = summary_mean_durations.pivot(values='value', columns='kernel', index='variable').reset_index().rename(columns={'variable':'metric'})[['metric'] + jank.wa_paths]
    summary_parts.append(summary_mean_durations)

if 'jank_percs' in jank.summary:
    summary_jank_percs = jank.summary['jank_percs'].copy()
    summary_jank_percs['perc_diff'] = summary_jank_percs['perc_diff'].apply(lambda s: f"({s})")
    summary_jank_percs['value'] = summary_jank_percs['value'] + " " + summary_jank_percs['perc_diff']
    summary_jank_percs = summary_jank_percs.pivot(values='value', columns='kernel', index='variable').reset_index().rename(columns={'variable':'metric'})[['metric'] + jank.wa_paths]
    summary_parts.append(summary_jank_percs)

if 'max_duration' in jank.summary:
    summary_max_durations = jank.summary['max_duration'].copy()
    summary_max_durations = summary_max_durations.pivot(values='value', columns='kernel', index='variable').reset_index().rename(columns={'variable':'metric'})[['metric'] + jank.wa_paths]
    summary_parts.append(summary_max_durations)

if 'power_usage' in jank.summary:
    summary_power_usage = jank.summary['power_usage'].copy().query("channel == 'CPU'")
    summary_power_usage['perc_diff'] = summary_power_usage['perc_diff'].apply(lambda s: f"({s})")
    summary_power_usage['value'] = summary_power_usage['value'] + " " + summary_power_usage['perc_diff']
    summary_power_usage['channel'] = 'CPU_total_power'
    summary_power_usage = summary_power_usage.pivot(values='value', columns='kernel', index='channel').reset_index().rename(columns={'channel':'metric'})[['metric'] + jank.wa_paths]
    summary_parts.append(summary_power_usage)

if 'overutilized_mean' in jank.analysis:
    summary_ou = jank.analysis['overutilized_mean'].copy()
    summary_ou['percentage'] = summary_ou['percentage'].apply(lambda x: f"{x}%")
    summary_ou = summary_ou.pivot(values='percentage', columns='wa_path', index='metric').reset_index()[['metric'] + jank.wa_paths]
    summary_parts.append(summary_ou)

if 'thermal' in jank.summary:
    summary_thermal = jank.summary['thermal'].copy()
    summary_thermal['perc_diff'] = summary_thermal['perc_diff'].apply(lambda s: f"({s})")
    summary_thermal['value'] = summary_thermal['value'] + " " + summary_thermal['perc_diff']
    summary_thermal = summary_thermal.pivot(values='value', columns='kernel', index='cluster').reset_index().rename(columns={'cluster':'metric'})[['metric'] + jank.wa_paths]
    summary_thermal['metric'] = "thermal (" + summary_thermal['metric'] + ")"
    summary_parts.append(summary_thermal)

if 'wakeup_latency' in jank.summary:
    summary_wakeup_latency = jank.summary['wakeup_latency'].copy()
    summary_wakeup_latency['perc_diff'] = summary_wakeup_latency['perc_diff'].apply(lambda s: f"({s})")
    summary_wakeup_latency['comm'] = "latency (" + summary_wakeup_latency['comm'] + ")"
    summary_wakeup_latency['value'] = summary_wakeup_latency['value'] + " " + summary_wakeup_latency['perc_diff']
    summary_wakeup_latency = summary_wakeup_latency.pivot(values='value', columns='kernel', index='comm').reset_index().rename(columns={'comm':'metric'})[['metric'] + jank.wa_paths]
    summary_parts.append(summary_wakeup_latency)

if 'task_cpu_residency' in jank.summary:
    summary_task_cpu_residency = jank.summary['task_cpu_residency'].copy().query("cluster == 'total'")
    summary_task_cpu_residency['perc_diff'] = summary_task_cpu_residency['perc_diff'].apply(lambda s: f"({s})")
    summary_task_cpu_residency['value'] = summary_task_cpu_residency['value'] + " " + summary_task_cpu_residency['perc_diff']
    summary_task_cpu_residency = summary_task_cpu_residency.pivot(values='value', columns='kernel', index='comm').reset_index().rename(columns={'comm':'metric'})[['metric'] + jank.wa_paths]
    summary_task_cpu_residency['metric'] = "CPU residency (" + summary_task_cpu_residency['metric'] + ")"
    summary_parts.append(summary_task_cpu_residency)


summary = pd.concat(summary_parts).reset_index(drop=True)

print('Jankbench')
ptable(summary)